# Глубокая нейронная сеть для классификации изображений: на основе pytorch

# 1. Подготовка данных

In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from dnn_app_utils_v3 import load_data

train_x_orig, train_y, test_x_orig, test_y, classes = load_data()
print(train_x_orig.shape, train_y.shape, test_x_orig.shape, test_y.shape)

train_x = torch.from_numpy(train_x_orig.reshape(-1, 64 * 64 * 3)).type(torch.float32) / 255.
test_x = torch.from_numpy(test_x_orig.reshape(-1, 64 * 64 * 3)).type(torch.float32) / 255.
train_y = torch.from_numpy(train_y.reshape(-1, 1)).type(torch.float32)
test_y = torch.from_numpy(test_y.reshape(-1, 1)).type(torch.float32)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(209, 64, 64, 3) (1, 209) (50, 64, 64, 3) (1, 50)
torch.Size([209, 12288]) torch.Size([209, 1]) torch.Size([50, 12288]) torch.Size([50, 1])


In [2]:
train_dataset = TensorDataset(train_x, train_y)
test_dataset = TensorDataset(test_x, test_y)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)

# 2. Архитектура нейросети

In [3]:
import torch.nn as nn
n_x = 12288     # num_px * num_px * 3
n_h = 7
n_y = 1
model = nn.Sequential(nn.Linear(n_x, n_h), nn.ReLU(), nn.Linear(n_h, n_y), nn.Sigmoid())
model

Sequential(
  (0): Linear(in_features=12288, out_features=7, bias=True)
  (1): ReLU()
  (2): Linear(in_features=7, out_features=1, bias=True)
  (3): Sigmoid()
)

# 3. Loss & optimizer

In [4]:
criterion = nn.BCELoss()
learning_rate = 0.0075
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 4. Обучение (доделать.....)

In [5]:
# Train the model
num_epochs = 1000
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):    
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, loss.item()))

Epoch [100/1000], Loss: 0.1450
Epoch [200/1000], Loss: 0.0078
Epoch [300/1000], Loss: 0.0046
Epoch [400/1000], Loss: 0.0011
Epoch [500/1000], Loss: 0.0006
Epoch [600/1000], Loss: 0.0017
Epoch [700/1000], Loss: 0.0020
Epoch [800/1000], Loss: 0.0007
Epoch [900/1000], Loss: 0.0008
Epoch [1000/1000], Loss: 0.0011


In [6]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        predicted, _ = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.round() == labels.flatten()).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 76.0 %
